In [7]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import pandas as pd

gradient computation etc. not efficient for whole data set
 -> divide dataset into small batches

 --> DataLoader can do the batch computation for us

  - Implement a custom Dataset:
  - inherit Dataset
  - implement __init__ , __getitem__ , and __len__

In [5]:
class WineDataset(Dataset):

    def __init__(self):
        # Initialize data, download, etc.
        xy = pd.read_csv("C:\\Users\\nandi\\OneDrive\\Desktop\\wine.csv")
        self.n_samples = xy.shape[0]

        # here the first column is the class label, the rest are the features
         # Convert to NumPy and then to tensor
        self.x_data = torch.from_numpy(xy.iloc[:, 1:].values)  # Features
        self.y_data = torch.from_numpy(xy.iloc[:, [0]].values)  # Labels

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples


In [6]:
# create dataset
dataset = WineDataset()

# get first sample and unpack
first_data = dataset[0]
features, labels = first_data
print(features, labels)

tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03], dtype=torch.float64) tensor([1])


In [8]:
# Load whole dataset with DataLoader

train_loader = DataLoader(dataset=dataset,
                          batch_size=4,
                          shuffle=True,
                          num_workers=0)
# shuffle: shuffle data, good for training
# num_workers: faster loading with multiple subprocesses

# convert to an iterator and look at one random sample
dataiter = iter(train_loader)
data = next(dataiter)
features, labels = data
print(features, labels)

tensor([[1.3670e+01, 1.2500e+00, 1.9200e+00, 1.8000e+01, 9.4000e+01, 2.1000e+00,
         1.7900e+00, 3.2000e-01, 7.3000e-01, 3.8000e+00, 1.2300e+00, 2.4600e+00,
         6.3000e+02],
        [1.4100e+01, 2.1600e+00, 2.3000e+00, 1.8000e+01, 1.0500e+02, 2.9500e+00,
         3.3200e+00, 2.2000e-01, 2.3800e+00, 5.7500e+00, 1.2500e+00, 3.1700e+00,
         1.5100e+03],
        [1.1870e+01, 4.3100e+00, 2.3900e+00, 2.1000e+01, 8.2000e+01, 2.8600e+00,
         3.0300e+00, 2.1000e-01, 2.9100e+00, 2.8000e+00, 7.5000e-01, 3.6400e+00,
         3.8000e+02],
        [1.2670e+01, 9.8000e-01, 2.2400e+00, 1.8000e+01, 9.9000e+01, 2.2000e+00,
         1.9400e+00, 3.0000e-01, 1.4600e+00, 2.6200e+00, 1.2300e+00, 3.1600e+00,
         4.5000e+02]], dtype=torch.float64) tensor([[2],
        [1],
        [2],
        [2]])


In [9]:
# Dummy Training loop
num_epochs = 2
total_samples = len(dataset)
n_iterations = math.ceil(total_samples/4)
print(total_samples, n_iterations)
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        
        # here: 178 samples, batch_size = 4, n_iters=178/4=44.5 -> 45 iterations
        # Run your training process
        if (i+1) % 5 == 0:
            print(f'Epoch: {epoch+1}/{num_epochs}, Step {i+1}/{n_iterations}| Inputs {inputs.shape} | Labels {labels.shape}')


178 45
Epoch: 1/2, Step 5/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 10/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 15/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 20/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 25/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 30/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 35/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 40/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 1/2, Step 45/45| Inputs torch.Size([2, 13]) | Labels torch.Size([2, 1])
Epoch: 2/2, Step 5/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 2/2, Step 10/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 2/2, Step 15/45| Inputs torch.Size([4, 13]) | Labels torch.Size([4, 1])
Epoch: 2/2, Step 20/45| Inputs torch.Size([4, 1

# Dataset Transforms

In [3]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np

class WineDataset(Dataset):

    def __init__(self, transform=None):

        xy = pd.read_csv("C:\\Users\\nandi\\OneDrive\\Desktop\\wine.csv", dtype=np.float32)
        self.n_samples = xy.shape[0]

        self.x_data = xy.iloc[:, 1:].values  # features
        self.y_data = xy.iloc[:, [0]].values  # labels (2D to keep shape consistent)

        self.transform = transform

    def __getitem__(self, index):
        # Extract one sample
        x = self.x_data[index]
        y = self.y_data[index]

        sample = (x, y)

        if self.transform:
            sample = self.transform(sample)
        return sample

    def __len__(self):
        return self.n_samples


In [1]:
# Custom Transforms
# implement __call__(self, sample)
class ToTensor:
    # Convert ndarrays to Tensors
    def __call__(self, sample):
        inputs, targets = sample
        return torch.from_numpy(inputs), torch.from_numpy(targets)

class MulTransform:
    # multiply inputs with a given factor
    def __init__(self, factor):
        self.factor = factor

    def __call__(self, sample):
        inputs, targets = sample
        inputs *= self.factor
        return inputs, targets

In [4]:
print('Without Transform')
dataset = WineDataset()
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)

Without Transform
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
 2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03] [1.]


In [5]:
print('\nWith Tensor Transform')
dataset = WineDataset(transform=ToTensor())
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)


With Tensor Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
        3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
        1.0650e+03]) tensor([1.])


In [8]:
print('\nWith Tensor and Multiplication Transform')
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(2)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)


With Tensor and Multiplication Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([2.8460e+01, 3.4200e+00, 4.8600e+00, 3.1200e+01, 2.5400e+02, 5.6000e+00,
        6.1200e+00, 5.6000e-01, 4.5800e+00, 1.1280e+01, 2.0800e+00, 7.8400e+00,
        2.1300e+03]) tensor([1.])


In [9]:
print('\nWith Tensor and Multiplication Transform')
composed = torchvision.transforms.Compose([ToTensor(), MulTransform(4)])
dataset = WineDataset(transform=composed)
first_data = dataset[0]
features, labels = first_data
print(type(features), type(labels))
print(features, labels)


With Tensor and Multiplication Transform
<class 'torch.Tensor'> <class 'torch.Tensor'>
tensor([5.6920e+01, 6.8400e+00, 9.7200e+00, 6.2400e+01, 5.0800e+02, 1.1200e+01,
        1.2240e+01, 1.1200e+00, 9.1600e+00, 2.2560e+01, 4.1600e+00, 1.5680e+01,
        4.2600e+03]) tensor([1.])
